# 🚀 YOLOv5 工件检测 - 全自动训练

## 📊 数据信息
- **图片数量**：23张工件图片
- **标注框数**：371个标注框
- **类别数**：10个类别
- **平均密度**：16.1个标注框/图片

## 🏷️ 检测类别
轴承、法兰、螺栓、齿轮、支架、连接件、套筒、夹具、底座、轴套

## ⚡ 使用说明
1. 点击菜单：**运行时 → 全部运行**
2. 上传 `dataset.zip` 文件（当提示时）
3. 等待 30-60 分钟
4. 自动下载训练好的模型

---

In [ ]:
# ===== 步骤 1: 检查 GPU =====
print("🔍 检查 GPU 状态...\n")
!nvidia-smi
print("\n✅ GPU 检查完成")

In [ ]:
# ===== 步骤 2: 安装 YOLOv5 =====
print("📦 克隆 YOLOv5 仓库...")
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

print("\n📦 安装依赖包...")
!pip install -r requirements.txt -q

print("\n✅ YOLOv5 安装完成")

In [ ]:
# ===== 步骤 3: 上传数据集 =====
from google.colab import files
import zipfile
import os

print("📤 请上传 dataset.zip 文件...\n")
uploaded = files.upload()

print("\n📦 解压数据集...")
!unzip -q dataset.zip

print("\n✅ 数据集已准备就绪")

In [ ]:
# ===== 步骤 4: 验证数据集 =====
import os

train_images = len(os.listdir('dataset/images/train'))
train_labels = len(os.listdir('dataset/labels/train'))

print(f"📊 数据集统计：")
print(f"   - 训练图片：{train_images} 张")
print(f"   - 标注文件：{train_labels} 个")
print(f"\n📁 前5张图片：")
!ls dataset/images/train | head -5

if train_images == train_labels:
    print(f"\n✅ 数据集验证通过！")
else:
    print(f"\n⚠️ 警告：图片和标注数量不匹配")

In [ ]:
# ===== 步骤 5: 开始训练 =====
import time

print("🚀 开始训练...\n")
print("⏱️ 预计时间：30-60 分钟")
print("📊 训练参数：")
print("   - 图片大小：640x640")
print("   - 批次大小：16")
print("   - 训练轮数：100 epochs")
print("   - 早停耐心：20 epochs")
print("\n" + "="*50 + "\n")

start_time = time.time()

!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/yolov5/dataset/data.yaml \
  --weights yolov5s.pt \
  --cache \
  --project runs/train \
  --name workpiece \
  --patience 20 \
  --save-period 10 \
  --exist-ok

end_time = time.time()
duration = (end_time - start_time) / 60

print(f"\n✅ 训练完成！用时：{duration:.1f} 分钟")

In [ ]:
# ===== 步骤 6: 查看训练结果 =====
from IPython.display import Image, display
import matplotlib.pyplot as plt

print("📊 训练结果可视化\n")

print("=" * 50)
print("📈 训练曲线")
print("=" * 50)
display(Image('runs/train/workpiece/results.png', width=900))

print("\n" + "=" * 50)
print("📊 混淆矩阵")
print("=" * 50)
display(Image('runs/train/workpiece/confusion_matrix.png', width=700))

print("\n" + "=" * 50)
print("🏷️ 标注分布")
print("=" * 50)
display(Image('runs/train/workpiece/labels.jpg', width=700))

print("\n" + "=" * 50)
print("🎯 预测示例")
print("=" * 50)
try:
    display(Image('runs/train/workpiece/val_batch0_pred.jpg', width=900))
except:
    print("预测图片未生成")

In [ ]:
# ===== 步骤 7: 读取训练指标 =====
import pandas as pd

print("📊 最终训练指标\n")

try:
    results = pd.read_csv('runs/train/workpiece/results.csv')
    results.columns = results.columns.str.strip()
    
    last_epoch = results.iloc[-1]
    
    print(f"🎯 最佳性能指标：")
    print(f"   - mAP@0.5: {last_epoch['metrics/mAP_0.5']:.4f}")
    print(f"   - mAP@0.5:0.95: {last_epoch['metrics/mAP_0.5:0.95']:.4f}")
    print(f"   - Precision: {last_epoch['metrics/precision']:.4f}")
    print(f"   - Recall: {last_epoch['metrics/recall']:.4f}")
    print(f"   - 训练轮数: {int(last_epoch['epoch']) + 1}")
    
    print(f"\n📈 训练趋势：")
    print(results[['epoch', 'metrics/mAP_0.5', 'metrics/mAP_0.5:0.95']].tail(10))
    
except Exception as e:
    print(f"无法读取结果文件：{e}")
    print("请手动查看 runs/train/workpiece/results.csv")

In [ ]:
# ===== 步骤 8: 下载模型和结果 =====
from google.colab import files
import os

print("💾 准备下载文件...\n")

# 下载最佳模型
print("📥 下载最佳模型 (best.pt)...")
files.download('runs/train/workpiece/weights/best.pt')

# 下载最后一个模型
print("📥 下载最后模型 (last.pt)...")
files.download('runs/train/workpiece/weights/last.pt')

# 打包所有结果
print("\n📦 打包完整结果...")
!zip -r -q training_results.zip runs/train/workpiece

print("📥 下载完整结果 (training_results.zip)...")
files.download('training_results.zip')

print("\n✅ 所有文件下载完成！")
print("\n📦 已下载文件：")
print("   1. best.pt - 最佳模型权重")
print("   2. last.pt - 最后一轮模型权重")
print("   3. training_results.zip - 完整训练结果")

In [ ]:
# ===== 步骤 9: 测试模型（可选）=====
print("🧪 在训练集上测试模型...\n")

!python detect.py \
  --weights runs/train/workpiece/weights/best.pt \
  --source dataset/images/train \
  --conf 0.25 \
  --save-txt \
  --save-conf \
  --project runs/detect \
  --name test_results \
  --exist-ok

print("\n✅ 测试完成！")
print("📁 结果保存在：runs/detect/test_results/")

# 显示前3张检测结果
import os
from IPython.display import Image, display

print("\n🎯 检测结果示例：\n")
result_images = sorted(os.listdir('runs/detect/test_results'))[:3]
for img in result_images:
    if img.endswith(('.jpg', '.png')):
        print(f"📷 {img}")
        display(Image(f'runs/detect/test_results/{img}', width=600))
        print()

---

## 🎉 训练完成！

### 📦 已下载文件
1. **best.pt** - 最佳模型（用于生产环境）
2. **last.pt** - 最后一轮模型（用于继续训练）
3. **training_results.zip** - 完整训练结果

### 🚀 下一步
1. 使用 `best.pt` 进行推理检测
2. 在新图片上测试模型性能
3. 根据需要调整训练参数重新训练

### 📚 使用模型
```python
from ultralytics import YOLO

# 加载模型
model = YOLO('best.pt')

# 推理
results = model('your_image.jpg')
results[0].show()
```

---

**GitHub**: https://github.com/xuanspark/yolov5-workpiece-detection
